
**El método de Newton para intervalos**

Igual que en la clase anterior, supondremos que $f(x)$ es una función continuamente diferenciable y que existe un valor $x∗$ donde $f(x∗)=0$ (es decir, $x∗$ es una raíz de $f$ ). Aquí abordaremos de nuevo cómo encontrar las raíces de f dentro de un intervalo inicial $X$. Supondremos además que existe una extensión del intervalo $F′$ para la derivada $f′$, y que (por el momento) éste no contiene al cero: $0∉F′(X)$.

La clave del método de Newton para intervalos está en aplicar el teorema del valor medio, que nos asegura que para cualquier $x∈X$ se cumple que
$f(x)=f(x∗)+f′(ξ)⋅(x−x∗)$,

para algúna $ξ$ entre $x$ y $x∗$. Aquí, $ξ$ es un valor desconocido, pero podemos utilizar el hecho de que está contenido en el intervalo: $ξ∈X$.

Por lo tanto, obtenemos
$x∗=x−f(x)f′(ξ)∈x−f(x)F′(X)=:N(X,x)$,

donde hemos definido un operador $N$ que actúa sobre un intervalo y cualquier punto en el intervalo.

Si suponemos que $x∗∈X$, entonces $x∗∈N(X,x)∩X$ para toda $x∈X$.

Una elección particular es $x=m:=mid(X)$, el punto medio del intervalo $X$. Entonces obtenemos el llamado operador de Newton para intervalos:
$N(X):=N(X,m)=m−f(m)/F′(X)$.

Nota que cuando implementamos esto en la computadora, en general $f(m)$ no se podrá calcular exactamente, por lo cual es necesario convertir $m$ en un intervalo $M:=[m,m]$, y usar la extensión natural $F(M)$, así que tenemos finalmente
$N(X):=M−F(M)/F′(X)$,

donde ahora todos son intervalos.

Ahora podemos definir una sucesión de intervalos a partir de un intervalo inicial $X_0$, dada por $X_{k+1}:=Xk∩N(Xk)$. Por construcción, si $x∗∈X_0$ entonces $x∗∈Xk$ para toda $k$. Entonces, si $X_0$ contiene a una raíz, la raíz se mantiene dentro de la secuencia de intervalos, que de hecho forman una secuencia anidada que converge a $x∗$. Entonces, si controlamos que de alguna manera el diámetro de los intervalos $x_k$ disminuya, obtendremos cotas precisas para $x∗$. Esto es el contenido del Teorema del método de Newton para intervalos.

Más aún, si $f(x)$ es doblemente continua y diferenciable y $N(X)$ está bien definida en el intervalo $X$, se puede demostrar que:

$N(X)∩X=∅$, es decir, $N(X)$ no se intersecta con $X$, entonces $X$ no contiene ninguna raíz de $f$.

Si $N(X)⊆X$, entonces $X$ contiene exactamente una raíz de $f$.



**[1]** Escribe una función que calcule el operador de Newton para un intervalo $X$, dadas $f(x)$ y $f′(x)$.

In [1]:
using Intervalos

    ^(Intervalo,Real) at /home/juser/Intervalos.jl:116
is ambiguous with: 
    ^(Any,Integer) at intfuncs.jl:86.
To fix, define 
    ^(Intervalo,Integer)
before the new definition.


Primero necesito crear una funcion para interseccion:

In [30]:
#Intersección
function interseccion(A::Intervalo,B::Intervalo)
    C=Intervalo(0.0,0.1)
    #Primero los casos en los que no existe la interseccion
    if A.sup < B.inf
        println("No existe intersección entre los intervalos dados")
        return C="Vacio"
    elseif B.sup < A.inf
        println("No existe intersección entre los intervalos dados")
        return C="Vacio"
        #Despues evaluamos los casos de interseccion que tenemos
        elseif A.sup > B.inf && A.sup < B.sup && A.inf < B.inf
        C.inf=B.inf
        C.sup=A.sup
        return C
        elseif B.sup > A.inf && B.sup < A.sup && B.inf < A.inf
        C.inf=A.inf
        C.sup=B.sup
        return C
    elseif B.inf > A.inf && B.sup < A.sup
        C=B
        return C
    elseif A.inf > B.inf && A.sup < B.sup
        C=A
        return C
    end
end

interseccion (generic function with 1 method)

Pruebo que tal trabaja mi nueva funcion:

In [31]:
#Probando mi función para intersección
A=Intervalo(2.5,3.3)
B=Intervalo(3.5,4)
interseccion(A,B)#Funciona!

No existe intersección entre los intervalos dados


"Vacio"

Intentamos construir la funcion, primero probe con un ciclo for donde establezco el numero de iteraciones de manera arbitraria.

In [4]:
#Parametros f(x)=x^2-7 y f´(x)=2x
#Sabemos que existe una raiz en 2.646 (aprox)
X=Intervalo(1,3)
m=(X.inf+X.sup)/2
M=Intervalo(m,m)
N=M
for i=1:3
    N=M-(M^2.0-7)/(X^2.0)#Esta es la expresion relevante de Newton para intervalos
    X=interseccion(N,X)#Calculamos la interseccion
    m=(X.inf+X.sup)/2#Recalculamos el parametro m
    M=Intervalo(m,m)#Recalculamos el intervalo M
end
X

Intervalo(2.64685738516134941474000630279306278680451214313507080078125e+00 with 256 bits of precision,2.6468782050416227347340580866585924013634212315082550048828125e+00 with 256 bits of precision)

Que tan bueno es el resultado obtenido?

In [5]:
x=(X.inf+X.sup)/2
f=x^2.0-7

5.909124745402470735126643561454311258239854960353742384478378329834065297742794e-03 with 256 bits of precision

Obtuvimos 3 decimales de precision para el mismo numero de iteraciones. Ya estoy listo para implementar la funcion.

In [6]:
#Ahora creamos la funcion
function NewInt(F,G,X::Intervalo,err)#Parametros de entrada, función, derivada, intervalo y error
    m=(X.inf+X.sup)/2
    M=Intervalo(m,m)
    N=M
    c=1
    while c >= err
        N=M-F(M)/G(X)
        X=interseccion(N,X)
        m=(X.inf+X.sup)/2
        M=Intervalo(m,m)
        c=abs(X.sup-X.inf)
    end
    X
end

NewInt (generic function with 1 method)

Probamos la funcion, la diferencia es que ahora podemos introducir un error.

In [7]:
#Probamos para f(x)=x^2-7 y g(x)=2x
#X=Intervalo(1,3)
F(M::Intervalo)=M^2.0-7
G(X::Intervalo)=X*2.0
X=Intervalo(1,3)
NewInt(F,G,X,0.001)

Intervalo(2.6457475562796209148617998518915328531875275075435638427734375e+00 with 256 bits of precision,2.645755514705882415101012572478111906093545258045196533203125e+00 with 256 bits of precision)

In [8]:
X=Intervalo(2.6457475562796209148617998518915328531875275075435638427734375,2.645755514705882415101012572478111906093545258045196533203125)
x=(X.inf+X.sup)/2
f=x^2.0-7
#Funciona

1.187562253149143878341874962438776496885721556976152194808060702513330397778191e-06 with 256 bits of precision

Mejoramos la precision en 3 decimales mas para un total de 6, esto dependerá siempre del error.

**[2]** Implementa el método de Newton para intervalos para encontrar las raíces de $f(x)=x^3−1$ a partir de $X=[−3,3]$. Muestra gráficamente la implementación del método. En cada iteración subsecuente del método, ilustra qué le pasa al diámetro de la refincación del intervalo que vas obteniendo.


In [10]:
#Ahora probamos para f(x)=x^3-1 y g(x)=3x^2
#Parte positiva del intervalo
F(M::Intervalo)=M^3.0-1
G(X::Intervalo)=(X^2.0)*3
X=Intervalo(-3,3)
NewInt(F,G,X,0.0001)

Intervalo(9.99998791591044934069787419872454847791232168674468994140625e-01 with 256 bits of precision,1.0000014625886583745210424434279872230035834945738315582275390625e+00 with 256 bits of precision)

In [11]:
F(M::Intervalo)=(M^2.0)*(1/8)-1
G(X::Intervalo)=(X)*(1/4)
X=Intervalo(-3,-eps(BigFloat))
NewInt(F,G,X,0.0001)

Intervalo(-2.8284385326955462287325249182146080784150399267673492431640625e+00 with 256 bits of precision,-2.82841843430375746402949399538329089409671723842620849609375e+00 with 256 bits of precision)

In [12]:
#Ahora probamos para f(x)=x^3-1 y g(x)=3x^2
#Parte negativa del intervalo
F(M::Intervalo)=M^3.0-1
G(X::Intervalo)=(X^2.0)*3
X=Intervalo(-3,-eps(BigFloat))
NewInt(F,G,X,0.0001)

No existe intersección entre los intervalos dados


LoadError: type Nothing has no field inf
while loading In[12], in expression starting on line 6

In [9]:
#Ahora probamos para f(x)=x^3-1 y g(x)=3x^2
#Parte positiva del intervalo
F(M::Intervalo)=M^3.0-1
G(X::Intervalo)=(X^2.0)*3
X=Intervalo(0.00001,3)
NewInt(F,G,X,0.0001)

Intervalo(9.999987651812378688042336205565874251988134346902370452880859375e-01 with 256 bits of precision,1.00000150686430814249427978523954152478836476802825927734375e+00 with 256 bits of precision)

Dividiendo si encontró la raiz que estoy buscando. ($x=1$)

In [10]:
#Ahora probamos para f(x)=x^2+1 y g(x)=2x
#Parte negativa del intervalo
F(M::Intervalo)=M^2.0+1
G(X::Intervalo)=X*2.0
X=Intervalo(-3,-0.1)
NewInt(F,G,X,0.01)

No existe intersección entre los intervalos dados


LoadError: type Nothing has no field inf
while loading In[10], in expression starting on line 6

In [11]:
#Ahora probamos para f(x)=x^2+1 y g(x)=2x
#Parte positiva del intervalo
F(M::Intervalo)=M^2.0+1
G(X::Intervalo)=X*2.0
X=Intervalo(0.1,3)
NewInt(F,G,X,0.01)

No existe intersección entre los intervalos dados


LoadError: type Nothing has no field inf
while loading In[11], in expression starting on line 6

Obviamente no se encontro la raiz para ninguno de los intervalos probados. (Por teorema)

In [27]:
#¿El intervalo contiene al cero?
X=Intervalo(-3,3)
if 0 > X.inf && 0 < X.sup
    println("El intervalo dado cotiene al cero")
    X=[Intervalo(X.inf,-eps(BigFloat));Intervalo(eps(BigFloat),X.sup)]
else
    X
end

El intervalo dado cotiene al cero


2-element Array{Intervalo,1}:
 Intervalo(-3e+00 with 256 bits of precision,-1.727233711018888925077270372560079914223200072887256277004740694033718360632485e-77 with 256 bits of precision)
 Intervalo(1.727233711018888925077270372560079914223200072887256277004740694033718360632485e-77 with 256 bits of precision,3e+00 with 256 bits of precision)  

Probando para encontrar la raiz de $2$.

In [13]:
F(M::Intervalo)=M^2.0-2
G(X::Intervalo)=(X*2)
X=Intervalo(0.1,3)
NewInt(F,G,X,0.0001)

Intervalo(1.41421100867420163928189769109167173155583441257476806640625e+00 with 256 bits of precision,1.414216921726997716439455654580115151475183665752410888671875e+00 with 256 bits of precision)

In [14]:
sqrt(2)-1.41421100867420163928189769109167173155583441257476806640625#Seis decimales de precision

2.553698893459355e-6

Finalmente va la función que parte al Intervalo en dos secciones cuando este contiene al cero y encuentra las respectivas raices:

In [48]:
#Ahora creamos la nueva funcion que divide el intervalo cuando este contiene al cero
function NewInt2(F,G,X::Intervalo,err)#Parametros de entrada, función, derivada, intervalo y error
    m=(X.inf+X.sup)/2
    M=Intervalo(m,m)
    N=M
    c=1
    if 0 > X.inf && 0 < X.sup
        println("El intervalo dado contiene al cero")
        X=[Intervalo(X.inf,-eps(BigFloat));Intervalo(eps(BigFloat),X.sup)]
        Xn1=X[1]
        Xn2=X[2]
        mn1=(Xn1.inf+Xn1.sup)/2
        Mn1=Intervalo(mn1,mn1)
        mn2=(Xn2.inf+Xn2.sup)/2
        Mn2=Intervalo(mn2,mn2)
        Nn1=Mn1
        Nn2=Mn2
        cn1=c
        cn2=c
        while cn1 >= err && typeof(Xn1) == Intervalo
            Nn1=Mn1-F(Mn1)/G(Xn1)
            Xn1=interseccion(Nn1,Xn1)
            if Xn1 == "Vacio"
                Xn1="Vacio"
            else
                mn1=(Xn1.inf+Xn1.sup)/2
                Mn1=Intervalo(mn1,mn1)
                cn1=abs(Xn1.sup-Xn1.inf)
            end
        end
        if Xn1 == "Vacio"
            println("La parte negativa del intervalo evaluado no contiene raices")
        else
            println("La parte negativa del intervalo evaluado contiene una raíz en:")
            println(Xn1)
        end
        while cn2 >= err && typeof(Xn2) == Intervalo
            Nn2=Mn2-F(Mn2)/G(Xn2)
            Xn2=interseccion(Nn2,Xn2)
            if Xn2 == "Vacio"
                Xn2="Vacio"
            else
                mn2=(Xn2.inf+Xn2.sup)/2
                Mn2=Intervalo(mn2,mn2)
                cn2=abs(Xn2.sup-Xn2.inf)
            end
        end
        if Xn2 == "Vacio"
            println("La parte positiva del intervalo evaluado no contiene raices")
        else
            println("La parte positiva del intervalo evaluado contiene una raíz en:")
            println(Xn2)
        end
    else
        println("El intervalo dado no contiene al cero")
        while c >= err
            N=M-F(M)/G(X)
            X=interseccion(N,X)
            m=(X.inf+X.sup)/2
            M=Intervalo(m,m)
            c=abs(X.sup-X.inf)
        end
        println("El intervalo evaluado contiene una raíz en:")
        println(X)
    end
end

NewInt2 (generic function with 1 method)

In [49]:
#Ahora probamos para f(x)=x^3-1 y g(x)=3x^2
#Parte positiva del intervalo
F(M::Intervalo)=M^3.0+1
G(X::Intervalo)=(X^2.0)*3
X=Intervalo(-3,3)
NewInt2(F,G,X,0.0001)

El intervalo dado contiene al cero
La parte negativa del intervalo evaluado contiene una raíz en:
Intervalo(-1.0000015068486091409879525093717944628224358893930912017822265625e+00 with 256 bits of precision,-9.999987651967248599425321098355112781064235605299472808837890625e-01 with 256 bits of precision)
No existe intersección entre los intervalos dados
La parte positiva del intervalo evaluado no contiene raices


In [50]:
#Ahora probamos para f(x)=x^2-1 y g(x)=2x
#Parte positiva del intervalo
F(M::Intervalo)=M^2.0-1
G(X::Intervalo)=(X)*2
X=Intervalo(-3,3)
NewInt2(F,G,X,0.0001)

El intervalo dado contiene al cero
La parte negativa del intervalo evaluado contiene una raíz en:
Intervalo(-1.00000003448786996642879123186296863590172279145917855203151702880859375e+00 with 256 bits of precision,-9.9999996659255086112027597722122507217790143840829841792583465576171875e-01 with 256 bits of precision)
La parte positiva del intervalo evaluado contiene una raíz en:
Intervalo(9.9999996659255086112027597722122507217790143840829841792583465576171875e-01 with 256 bits of precision,1.00000003448786996642879123186296863590172279145917855203151702880859375e+00 with 256 bits of precision)


Funciona correctamente.

**[3]** Usando tu implementación, demuestra que en el mismo intervalo $X=[−3,3 ]$ que $g(x)=x^2+1$ no tiene ninguna raíz. Ilustra gráficamente esto.

Hint: En el caso en que $0∈F′(x)$ las hipótesis del teorema no se cumplen. En ese caso hay que dividir el intervalo (bisección por ejemplo) y tratar de que las hipótesis del teorema se cumplan, para poder aplicarlo. Extiende tus funciones para que incluyan esta situación. En ese caso, cada subdivisión debe ser probada si puede o no tener una raíz.

In [53]:
#Ahora probamos para f(x)=x^2+1 y g(x)=2x
#Todo el intervalo
F(M::Intervalo)=M^2.0+1
G(X::Intervalo)=X*2.0
X=Intervalo(-3,3)
NewInt2(F,G,X,0.0001)

El intervalo dado contiene al cero
No existe intersección entre los intervalos dados
La parte negativa del intervalo evaluado no contiene raices
No existe intersección entre los intervalos dados
La parte positiva del intervalo evaluado no contiene raices


Funciona correctamente.

**[4]** Considera la familia de polinomios de Wilkinson definidos por $W_n(x)=∏ni=1(x−i)$. Partiendo de un intervalo simétrico alrededor de cero, implementa el método de Newton para encontrar sus raíces, utilizando diferenciación automática.
División extendida


**[5]** Supón que $F'(X)$ sea un intervalo, digamos $F'(X)=[−a,b]$, que contiene $0$ (con $a,b>0$).

(i) Definiendo $1/A$ como el conjunto $1/x:x∈A$, evalúa $1/F'(X)$.

(ii) Define una función que implementa esta "división extendida" de intervalos (o, más bien, inversa extendida).

**R:** A continuación genero la función de división extendida y hago algunas pruebas.

In [54]:
FX=Intervalo(-3,2)

Intervalo(-3e+00 with 256 bits of precision,2e+00 with 256 bits of precision)

In [55]:
#Division extendida
function extend(A::Intervalo)
    D=Intervalo(0,0)
    C=[Intervalo(0,0);Intervalo(0,0)]
    if (A.inf <= 0.0) & (A.sup > 0.0)
        C=[Intervalo(-Inf,1/A.inf);Intervalo(1/A.sup,Inf)]
        return C
    else
        D=Intervalo(1/A.sup,1/A.inf)
        return D
    end
end

extend (generic function with 1 method)

In [56]:
EX=extend(FX)

2-element Array{Intervalo,1}:
 Intervalo(-inf with 256 bits of precision,-3.33333333333333314829616256247390992939472198486328125e-01 with 256 bits of precision)
 Intervalo(5e-01 with 256 bits of precision,inf with 256 bits of precision)                                                        

In [46]:
EX[2]

Intervalo(5e-01 with 256 bits of precision,inf with 256 bits of precision)

In [41]:
X=Intervalo(1,3)

Intervalo(1e+00 with 256 bits of precision,3e+00 with 256 bits of precision)

In [42]:
X*EX[2]

Intervalo(5e-01 with 256 bits of precision,inf with 256 bits of precision)